In [1]:
import pandas as pd
import numpy as np

# Median Zip Income

The data for median zip income is separated across several csv files. Each csv file corresponds to a different year with each row a different geographical region.


## Initial goals:

Load all csv files, extract total household income and margin of error. Append to a master table.

In [2]:
#String prefix and suffix for loading files as well as years
prefix_filenames = 'ACSST5Y'
sufix_filenames = '.S1901_data_with_overlays_2021-12-16T085619.csv'
sufix_meta = '.s1901_metadata_2021-12-16T085619.csv'

years = ['2011','2012','2013','2014','2015','2016','2017','2018','2019']

#These are the columns to read in the meta file. It will provide the names in the data file to read from
meta_columns = ['S1901_C01_001E','S1901_C01_001M','NAME'] 


In [3]:
#These are the column names that will be assigned to the full table (including year). 
#Years are separate, since column_names will be used to rename each CSV as it's read.
column_names = ['Geo_Area','Household_Total','Margin_of_Error']
full_df = pd.DataFrame(columns=column_names.append('Year'))

In [4]:
#meta file is read first to ID the columns that need to be read.
#These columns are read, renamed, and the appended to the master file.
for yr in years:
    df_meta = pd.read_csv(prefix_filenames+yr+sufix_meta)
    cols_to_extract = df_meta.id[df_meta[df_meta['GEO_ID'].isin(meta_columns)].index].values
    
    rename_dict = dict(zip(cols_to_extract,column_names))
    
    
    df = pd.read_csv(prefix_filenames+yr+sufix_filenames,skiprows=1,usecols=cols_to_extract,low_memory=False)
    df.rename(rename_dict,axis=1,inplace=True)
    df['Year'] = int(yr)
        
    full_df = full_df.append(df)

In [5]:
full_df

,Geo_Area,Household_Total,Margin_of_Error,Year
0,ZCTA5 00601,5210.0,212.0,2011
1,ZCTA5 00602,13500.0,297.0,2011
2,ZCTA5 00603,17958.0,439.0,2011
3,ZCTA5 00606,1679.0,163.0,2011
4,ZCTA5 00610,9288.0,286.0,2011
...,...,...,...,...
33115,ZCTA5 99923,14.0,20.0,2019
33116,ZCTA5 99925,317.0,41.0,2019
33117,ZCTA5 99926,487.0,41.0,2019
33118,ZCTA5 99927,0.0,9.0,2019


## Identification of NaNs

Our first goal is to identify how the NaNs are located in the table. Are they entire geographical regions, or just particular years for different geo regions?

In [6]:
#First we pivot over the year, to identify geographical areas that have values missing the resulting output is a series with the geographical areas that have NaNs at all.
pivot_df = full_df.pivot(index = 'Year',columns='Geo_Area')
area_nan_bool = pivot_df.isna().any().Household_Total
nan_areas = pd.Series(area_nan_bool[area_nan_bool].index)

In [7]:
#Now pivot over the geographical area so that we can identify their time series NaNs and see where they need to be filled
pivot_df = full_df.pivot(index = 'Geo_Area',columns = 'Year')
pivot_df.loc[nan_areas,:]

Household_Total                                                  \
Year                   2011    2012    2013    2014    2015    2016    2017   
Geo_Area                                                                      
ZCTA5 00801             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00802             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00820             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00823             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00824             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00830             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00831             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00840             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00841             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00850             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 00851             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 87012           114.0   126.0   104.0   118.0    92.0    89.0   100.0   
ZCTA5 87013          1452.0  1451.0  1365.0  1415.0  1349.0  1428.0  1427.0   
ZCTA5 87017           186.0   157.0   151.0   173.0   157.0   127.0   143.0   
ZCTA5 87029           157.0   111.0   110.0    91.0   117.0   123.0   132.0   
ZCTA5 87046            11.0    16.0    22.0    25.0    23.0    33.0    36.0   
ZCTA5 87064            92.0    70.0    69.0    60.0    70.0    54.0    79.0   
ZCTA5 87412           409.0   329.0   355.0   329.0   298.0   311.0   339.0   
ZCTA5 87419           159.0   117.0   101.0   124.0   139.0   122.0   140.0   
ZCTA5 87510           286.0   247.0   311.0   222.0   278.0   313.0   364.0   
ZCTA5 87511           894.0   817.0   748.0   786.0   797.0   729.0   745.0   
ZCTA5 87515           175.0   136.0   118.0   104.0   148.0   135.0   141.0   
ZCTA5 87516            55.0    50.0    60.0    42.0    30.0    36.0    35.0   
ZCTA5 87518            19.0    17.0    30.0    41.0    37.0    40.0    24.0   
ZCTA5 87520           773.0   646.0   590.0   474.0   440.0   385.0   509.0   
ZCTA5 87521           304.0   392.0   374.0   378.0   513.0   568.0   341.0   
ZCTA5 87522          1274.0  1250.0  1195.0  1100.0   994.0   824.0   977.0   
ZCTA5 87523           196.0   167.0    93.0   131.0   118.0   146.0   152.0   
ZCTA5 87527           471.0   410.0   363.0   270.0   318.0   306.0   299.0   
ZCTA5 87528           960.0   973.0  1013.0   972.0   928.0   947.0   856.0   
ZCTA5 87530           305.0   373.0   342.0   347.0   373.0   345.0   224.0   
ZCTA5 87531           240.0   199.0   214.0   172.0   172.0   214.0   158.0   
ZCTA5 87532          6786.0  6858.0  6868.0  7000.0  6721.0  6666.0  6615.0   
ZCTA5 87533            69.0    56.0    62.0    64.0    52.0    58.0    59.0   
ZCTA5 87537          1067.0  1083.0  1160.0  1153.0   930.0   983.0   957.0   
ZCTA5 87539           152.0   172.0   164.0   164.0   109.0    59.0    21.0   
ZCTA5 87548           356.0   325.0   307.0   329.0   292.0   289.0   362.0   
ZCTA5 87549           340.0   415.0   455.0   357.0   404.0   409.0   257.0   
ZCTA5 87551           145.0   193.0   154.0   148.0   157.0   178.0   153.0   
ZCTA5 87554            40.0    40.0    23.0    23.0    13.0     0.0     0.0   
ZCTA5 87566           953.0   978.0   929.0   961.0   948.0   900.0   831.0   
ZCTA5 87575           292.0   253.0   273.0   283.0   253.0   368.0   383.0   
ZCTA5 87577            20.0    23.0    18.0    56.0   101.0   129.0   173.0   
ZCTA5 87578           267.0   427.0   461.0   415.0   215.0   217.0   158.0   
ZCTA5 87581            73.0    52.0    42.0    15.0     0.0     0.0     0.0   
ZCTA5 87582           590.0   583.0   540.0   483.0   465.0   288.0   230.0   
ZCTA5 96799             NaN     NaN     NaN     NaN     NaN     NaN     NaN   
ZCTA5 96910             NaN     NaN     NaN     NaN     NaN   

What we can observe from this is that there are some geographical areas that did not contain any data across any year, and there are other geographical areas that only had NaNs in 2018. Thus, we will need two separate methods to address this aspect.

1) For those that have data only missing in 2018, we can interpolate between 2017 and 2019.
2) For those missing data across many years, we can either drop them or pad them. We will currently avoid dropping based on the comments from the guy's lecture last week. For padding we have two other options:


    a) We can interpolate by surrounding zip codes. This assumes some sort of geographical continuity in terms of    income, which is not a strong assumption (poor areas are near rich areas etc..). 
    
    
    b) We can find the average of the median income across year and pad these zip codes. However, this assumes that these geographical regions are near the mean income in the country. 
    
For now, we will do option (b) but we must consider doing option (a)

In [8]:
#Inteprolating for geo areas that are only missing 2018
#This method takes too much time. According to the internet, groupby -> interpolate is a very time consuming operation.
#The next cell will be the replacement approach.


#First, change the type of the year column to date time, in order to be able to perform time interpolation.
#Then, we set it as the index (again to perform time based interpolation) and group by the geo area. 

#full_df.Year = pd.to_datetime(full_df.Year,format='%Y')
#full_df.set_index('Year',inplace = True)
#full_df.groupby('Geo_Area').transform(lambda group: group.interpolate(method='time'))

#full_df.reset_index(inplace=True)
#full_df.Year = full_df.Year.dt.year

Since the above did not work due to time consumption, an unelegant way of solving this problem is to obtain the geographical regions in which both the year 2018 is defined and is a nan. Then, I think we can group by these geographical regions and then interpolate by time.

In [9]:
is_2018 = full_df.Year == 2018
is_nan = full_df.Household_Total.isna()

bad_2018_data = full_df[np.logical_and(is_2018,is_nan)]

In [10]:
full_df.Year = pd.to_datetime(full_df.Year,format= '%Y')
full_df.set_index('Year',inplace = True)

bad2018_filled = full_df[full_df.Geo_Area.isin(bad_2018_data.Geo_Area.values)].groupby('Geo_Area').apply(lambda x : x.interpolate(method='time'))


In [11]:
#Reset time index, and fill in with the indices of the "bad 2018 data"
full_df.reset_index(inplace=True)
full_df.Year = full_df.Year.dt.year
bad2018_filled.reset_index(inplace = True)
bad2018_filled.Year = bad2018_filled.Year.dt.year

full_df.loc[bad2018_filled.index,:] = bad2018_filled

To interpolate across those that have NaNs throughout their dataset I realized that the problem is that many of them only have one year defined (as a NaN) and then the other years are not even in the datasets. So we have two options:


     a) Create the absent rows and fill them with NaNs
     
     b) Drop these rows
     
For now we will go with (a), but we should talk about this.
   

To create the new rows, I figured that unpivoting the above pivoted tables should create them, since after all they existed during the pivot and should exist after the pivot. Thus, I grab each dataset (household total, margin of error) we melt these columns and then join them again into a new dataframe.

In [12]:
household_total_unpivoted = pd.melt(pivot_df.Household_Total,value_vars = [2011,2012,2013,2014,2015,2016,2017,2018,2019],ignore_index=False,value_name='Household_Total')
error_unpivoted = pd.melt(pivot_df.Household_Total,value_vars = [2011,2012,2013,2014,2015,2016,2017,2018,2019],ignore_index=False,value_name='Margin_of_Error')

full_df  = pd.merge(household_total_unpivoted.reset_index(), error_unpivoted.reset_index(),on= ['Geo_Area','Year'])

Once these absent rows have been created, we can proceed by finding the group average and then filling in those columns that have NaNs with the averages. 

In [13]:
#Interpolating across all years. First estimate the man household total and margin of error, 
# then generate a dictionary mapping the years to these means.
total_yearly_mean_dict = round(full_df.groupby('Year').mean()).Household_Total.to_dict()
error_yearly_mean_dict = round(full_df.groupby('Year').mean()).Margin_of_Error.to_dict()
year_mean_dict = {'Household_Total':total_yearly_mean_dict,'Margin_of_Error':error_yearly_mean_dict,'Geo_Area':{}}

In [14]:
#Set the year again as the index since we won't interpolate there, and need it as an index for the NaNs
full_df.set_index('Year',inplace = True)

In [15]:
#Then, to each column apply the dictionary to fill the NaNs
full_df = full_df.apply(lambda x: x.fillna(year_mean_dict[x.name]))

#reset the index
full_df.reset_index(inplace=True)

In [16]:
#Check again for NaNs
#First we pivot over the year, to identify geographical areas that have values missing the resulting output is a series with the geographical areas that have NaNs at all.
pivot_df = full_df.pivot(index = 'Year',columns='Geo_Area')
area_nan_bool = pivot_df.isna().any().Household_Total
nan_areas = pd.Series(area_nan_bool[area_nan_bool].index)

#Now pivot over the geographical area so that we can identify their time series NaNs and see where they need to be filled
pivot_df = full_df.pivot(index = 'Geo_Area',columns = 'Year')
pivot_df.loc[nan_areas,:]

Empty DataFrame
Columns: [(Household_Total, 2011), (Household_Total, 2012), (Household_Total, 2013), (Household_Total, 2014), (Household_Total, 2015), (Household_Total, 2016), (Household_Total, 2017), (Household_Total, 2018), (Household_Total, 2019), (Margin_of_Error, 2011), (Margin_of_Error, 2012), (Margin_of_Error, 2013), (Margin_of_Error, 2014), (Margin_of_Error, 2015), (Margin_of_Error, 2016), (Margin_of_Error, 2017), (Margin_of_Error, 2018), (Margin_of_Error, 2019)]
Index: []

In [17]:
full_df.to_csv('median_income_zip_code_clean.csv',index=False)

In [18]:
full_df.describe()

,Year,Household_Total,Margin_of_Error
count,298296.000000,298296.000000,298296.000000
mean,2015.000000,3579.167159,3579.167159
std,2.581993,5137.061541,5137.061541
min,2011.000000,0.000000,0.000000
25%,2013.000000,273.000000,273.000000
50%,2015.000000,1064.000000,1064.000000
75%,2017.000000,4936.000000,4936.000000
max,2019.000000,43607.000000,43607.000000
